In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit
from collections import Counter
from surprise import Dataset, Reader
from surprise import SVD

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_als_bpr_artist_hyper_pred"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_als_bpr_artist_hyper_target"
log_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_als_bpr_artist_hyper_log"
# df_path = '/Users/david/Documents/GitHub/YandexCup_RecSys/track_artists.csv'
# train_path = '/Users/david/Documents/GitHub/YandexCup_RecSys/train'
# pred_path = '/Users/david/Documents/GitHub/YandexCup_RecSys/ensemble_als_bpr_artist_hyper_pred'
# target_path = '/Users/david/Documents/GitHub/YandexCup_RecSys/ensemble_als_bpr_artist_hyper_target'

In [3]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")
    
    return mrr_score / len(y_true)

In [4]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 46.9 ms
Wall time: 56.8 ms


In [5]:
artist_coord = []

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(100000), size=25000, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            artist_arr = artist_arr[:-1]
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        artist_coord.extend(list_coord_user)
        
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 100000:
            break

  1%|▋                                                                        | 9999/1160084 [00:06<12:46, 1500.16it/s]


In [6]:
%%time

data = pd.DataFrame(artist_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')

CPU times: total: 2.5 s
Wall time: 2.49 s


In [7]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 469 ms
Wall time: 469 ms


In [8]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 797 ms
Wall time: 784 ms


In [9]:
%%time

algo = SVD(n_factors=300, 
            n_epochs=770, 
            biased=False, 
            init_mean=0, 
            init_std_dev=0.1, 
            lr_bu=0.01625, lr_bi=0.01625, lr_pu=0.01625, lr_qi=0.00005, 
            reg_bu=0.005, reg_bi=0.005, reg_pu=0.05, reg_qi=0.001,
            verbose=True)

print('algo.fit:')
algo.fit(trainset)

algo.fit:
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


Processing epoch 395
Processing epoch 396
Processing epoch 397
Processing epoch 398
Processing epoch 399
Processing epoch 400
Processing epoch 401
Processing epoch 402
Processing epoch 403
Processing epoch 404
Processing epoch 405
Processing epoch 406
Processing epoch 407
Processing epoch 408
Processing epoch 409
Processing epoch 410
Processing epoch 411
Processing epoch 412
Processing epoch 413
Processing epoch 414
Processing epoch 415
Processing epoch 416
Processing epoch 417
Processing epoch 418
Processing epoch 419
Processing epoch 420
Processing epoch 421
Processing epoch 422
Processing epoch 423
Processing epoch 424
Processing epoch 425
Processing epoch 426
Processing epoch 427
Processing epoch 428
Processing epoch 429
Processing epoch 430
Processing epoch 431
Processing epoch 432
Processing epoch 433
Processing epoch 434
Processing epoch 435
Processing epoch 436
Processing epoch 437
Processing epoch 438
Processing epoch 439
Processing epoch 440
Processing epoch 441
Processing ep

In [10]:
model = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                 learning_rate=0.075,
                                                 regularization=0.0025,
                                                 iterations=950,
                                                 verify_negative_samples=True,
                                                 random_state=42)   
print('model_bpr.fit:')
model.fit(user_item)

model_bpr.fit:


100%|███████████████████████████████████████████████| 950/950 [04:29<00:00,  3.53it/s, train_auc=99.99%, skipped=0.79%]


In [11]:
def N_opt(model, N=100, q_bpr=1):
    
    userid = val_id

    start_time = time.time()
    rec = model.recommend(userid, user_item_val, N=N, filter_already_liked_items=True)
    print("\n--- %s seconds model_bpr ---" % (time.time() - start_time))
    
    return rec, N

In [25]:
def optimize(algo,
             rec,
             N,
             coef_art=1,
             norm_all=True,
             preproc_model=None,
             preproc_algo=None):
        
    if preproc_model is None:
        model_score = rec[1]
    else:
        if norm_all:
            if preproc_model == 'minmax':
                model_score = minmax_scale(rec[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N))
            if preproc_model == 'standart':
                model_score = scale(rec[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N))    
        else:
            if preproc_model == 'minmax':
                model_score = minmax_scale(rec[1], feature_range=(0, 1), axis=1)   
            if preproc_model == 'standart':
                model_score = scale(rec[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i, val_i in tqdm(zip(range(rec[0].shape[0]), val_id)):
        df_final = pd.DataFrame({'trak_id': rec[0][i], 'model_score': model_score[i]})
        df_final = pd.merge(df_final, df, how="inner", left_on='trak_id', right_on=df.index, sort=False)
        df_final['artist_score'] = np.array([algo.predict(uid=val_i, iid=iid).est for iid in df_final.artistId.values])
        if preproc_algo == 'minmax':
            df_final['artist_score'] = minmax_scale(df_final.artist_score.values, feature_range=(0, 1), axis=0)
        if preproc_algo == 'standart':
            df_final['artist_score'] = scale(df_final.artist_score.values, with_mean=True, with_std=True, axis=0)
        df_final['final_score'] = df_final.model_score + df_final.artist_score * coef_art
        final_rec = df_final.sort_values(by='final_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

In [28]:
ls_first = []

In [29]:
with open(log_path, 'w') as f:
    for N in [100, 500, 1000]:
        rec, N = N_opt(model, N=N)
        for norm_all in [True, False]:
            for preproc_model in [None, 'minmax', 'standart']:
                for preproc_algo in [None, 'minmax', 'standart']:
                    if preproc_algo or preproc_model or norm_all:
                        for coef_art in [0.01, 0.1, 1, 10, 100]:
                            print(f'N: {N}, norm_all: {norm_all}, preproc_model: {preproc_model}, preproc_algo: {preproc_algo}, coef_art: {coef_art}')
                            mrr = optimize(algo,
                                           rec,
                                           N,
                                           coef_art=coef_art,
                                           norm_all=norm_all,
                                           preproc_model=preproc_model,
                                           preproc_algo=preproc_algo)
                            tup = (mrr, N, norm_all, preproc_model, preproc_algo, coef_art)
                            ls_first.append(tup)
                            log_str = ' '.join([str(i) for i in tup]) + '\n'
                            f.write(log_str)


--- 5.453796148300171 seconds model_bpr ---
N: 100, norm_all: True, preproc_model: None, preproc_algo: None, coef_art: 0.01


1000it [00:46, 21.54it/s]


MRR@100 = 0.0215
N: 100, norm_all: True, preproc_model: None, preproc_algo: None, coef_art: 0.1


1000it [00:47, 21.18it/s]


MRR@100 = 0.0212
N: 100, norm_all: True, preproc_model: None, preproc_algo: None, coef_art: 1


...

MRR@100 = 0.0302
N: 1000, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 10


1000it [00:49, 20.25it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


MRR@100 = 0.0229
N: 1000, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 100


1000it [00:49, 20.13it/s]


MRR@100 = 0.0225


In [30]:
data_first = pd.DataFrame(ls_first, columns=['mrr', 'N', 'norm_all', 'preproc_model', 'preproc_algo', 'coef_art'])
data_first.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,norm_all,preproc_model,preproc_algo,coef_art
111,0.030394,500,True,minmax,standart,0.1
167,0.030256,500,False,standart,standart,1.0
252,0.030242,1000,False,standart,standart,1.0
127,0.030175,500,True,standart,standart,1.0
196,0.030049,1000,True,minmax,standart,0.1
236,0.029615,1000,False,minmax,standart,0.1
212,0.029603,1000,True,standart,standart,1.0
142,0.029103,500,False,minmax,None,1.0
147,0.028999,500,False,minmax,minmax,1.0
227,0.028904,1000,False,minmax,None,1.0


In [ ]:
0.030394	500	True	minmax	standart	0.1

In [33]:
ls = []

In [34]:
for N in [250, 375, 500, 625, 750]:
    rec, N = N_opt(model, N=N)
    for norm_all in [True, False]:
        for preproc_model in ['minmax', 'standart']:
            for preproc_algo in ['minmax', 'standart']:
                if preproc_algo or preproc_model or norm_all:
                    for coef_art in [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1, 1.5, 2, 4]:
                        print(f'N: {N}, norm_all: {norm_all}, preproc_model: {preproc_model}, preproc_algo: {preproc_algo}, coef_art: {coef_art}')
                        mrr = optimize(algo,
                                       rec,
                                       N,
                                       coef_art=coef_art,
                                       norm_all=norm_all,
                                       preproc_model=preproc_model,
                                       preproc_algo=preproc_algo)
                        tup = (mrr, N, norm_all, preproc_model, preproc_algo, coef_art)
                        ls.append(tup)


--- 4.46681022644043 seconds model_bpr ---
N: 250, norm_all: True, preproc_model: minmax, preproc_algo: minmax, coef_art: 0.05


1000it [00:44, 22.23it/s]


MRR@100 = 0.0225
N: 250, norm_all: True, preproc_model: minmax, preproc_algo: minmax, coef_art: 0.075


1000it [00:44, 22.70it/s]


MRR@100 = 0.0246
N: 250, norm_all: True, preproc_model: minmax, preproc_algo: minmax, coef_art: 0.1


...

MRR@100 = 0.0290
N: 750, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 2


1000it [00:44, 22.33it/s]
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


MRR@100 = 0.0270
N: 750, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 4


1000it [00:43, 22.74it/s]

MRR@100 = 0.0249


In [35]:
data = pd.DataFrame(ls, columns=['mrr', 'N', 'norm_all', 'preproc_model', 'preproc_algo', 'coef_art'])
data.sort_values(by='mrr', ascending=False).head(50)

,mrr,N,norm_all,preproc_model,preproc_algo,coef_art
315,0.030590,625,False,standart,standart,0.750
194,0.030515,500,True,standart,standart,0.500
274,0.030513,625,True,standart,standart,0.500
189,0.030482,500,True,standart,minmax,4.000
12,0.030466,250,True,minmax,standart,0.100
396,0.030459,750,False,standart,standart,1.000
235,0.030434,500,False,standart,standart,0.750
354,0.030422,750,True,standart,standart,0.500
172,0.030394,500,True,minmax,standart,0.100
269,0.030384,625,True,standart,minmax,4.000


## Full valid

In [36]:
artist_coord = []

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(100000), size=25000, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        
        tracks_arr = np.array(tracks)
        tracks_arr = tracks_arr.astype(np.int32)
        artist_arr = np.array([df.at[track, 'artistId'] for track in tracks_arr])
        if idx in val_id:
            artist_arr = artist_arr[:-1]
        artist_cnt = Counter(artist_arr)
        list_coord_user = [(idx, i[0], i[1]) for i in artist_cnt.items()]
        artist_coord.extend(list_coord_user)
        
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 100000:
            break

  9%|██████▎                                                                  | 99999/1160084 [04:11<44:22, 398.22it/s]


In [37]:
%%time

data = pd.DataFrame(artist_coord, columns=['uid', 'iid', 'rating_row'])
data['rating_max'] = data.rating_row / data.groupby('uid')['rating_row'].transform('max')

CPU times: total: 26.7 s
Wall time: 26.7 s


In [38]:
%%time

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(data[['uid', 'iid', 'rating_max']], reader)
trainset = dataset.build_full_trainset()

CPU times: total: 6.64 s
Wall time: 6.64 s


In [39]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 5.19 s
Wall time: 5.17 s


In [40]:
%%time

algo = SVD(n_factors=300, 
            n_epochs=770, 
            biased=False, 
            init_mean=0, 
            init_std_dev=0.1, 
            lr_bu=0.01625, lr_bi=0.01625, lr_pu=0.01625, lr_qi=0.00005, 
            reg_bu=0.005, reg_bi=0.005, reg_pu=0.05, reg_qi=0.001,
            verbose=True)

print('algo.fit:')
algo.fit(trainset)

algo.fit:
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


Processing epoch 395
Processing epoch 396
Processing epoch 397
Processing epoch 398
Processing epoch 399
Processing epoch 400
Processing epoch 401
Processing epoch 402
Processing epoch 403
Processing epoch 404
Processing epoch 405
Processing epoch 406
Processing epoch 407
Processing epoch 408
Processing epoch 409
Processing epoch 410
Processing epoch 411
Processing epoch 412
Processing epoch 413
Processing epoch 414
Processing epoch 415
Processing epoch 416
Processing epoch 417
Processing epoch 418
Processing epoch 419
Processing epoch 420
Processing epoch 421
Processing epoch 422
Processing epoch 423
Processing epoch 424
Processing epoch 425
Processing epoch 426
Processing epoch 427
Processing epoch 428
Processing epoch 429
Processing epoch 430
Processing epoch 431
Processing epoch 432
Processing epoch 433
Processing epoch 434
Processing epoch 435
Processing epoch 436
Processing epoch 437
Processing epoch 438
Processing epoch 439
Processing epoch 440
Processing epoch 441
Processing ep

In [41]:
model = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                 learning_rate=0.075,
                                                 regularization=0.0025,
                                                 iterations=950,
                                                 verify_negative_samples=True,
                                                 random_state=42)   
print('model_bpr.fit:')
model.fit(user_item)

model_bpr.fit:


100%|███████████████████████████████████████████████| 950/950 [39:35<00:00,  2.50s/it, train_auc=99.76%, skipped=0.79%]


In [42]:
iter_data = pd.DataFrame(ls, columns=['mrr', 'N', 'norm_all', 'preproc_model', 'preproc_algo', 'coef_art'])
iter_arr = iter_data.sort_values(by='mrr', ascending=False).head(15).values
ls_full = []

In [43]:
for mrr, N, norm_all, preproc_model, preproc_algo, coef_art in iter_arr:
    print(f'N: {N}, norm_all: {norm_all}, preproc_model: {preproc_model}, preproc_algo: {preproc_algo}, coef_art: {coef_art}')

N: 625, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 0.75
N: 500, norm_all: True, preproc_model: standart, preproc_algo: standart, coef_art: 0.5
N: 625, norm_all: True, preproc_model: standart, preproc_algo: standart, coef_art: 0.5
N: 500, norm_all: True, preproc_model: standart, preproc_algo: minmax, coef_art: 4.0
N: 250, norm_all: True, preproc_model: minmax, preproc_algo: standart, coef_art: 0.1
N: 750, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 1.0
N: 500, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 0.75
N: 750, norm_all: True, preproc_model: standart, preproc_algo: standart, coef_art: 0.5
N: 500, norm_all: True, preproc_model: minmax, preproc_algo: standart, coef_art: 0.1
N: 625, norm_all: True, preproc_model: standart, preproc_algo: minmax, coef_art: 4.0
N: 500, norm_all: True, preproc_model: minmax, preproc_algo: minmax, coef_art: 0.5
N: 625, norm_all: False, preproc_model: standart, 

In [44]:
for mrr, N, norm_all, preproc_model, preproc_algo, coef_art in iter_arr:
    rec, N = N_opt(model, N=N)
    print(f'N: {N}, norm_all: {norm_all}, preproc_model: {preproc_model}, preproc_algo: {preproc_algo}, coef_art: {coef_art}')
    mrr = optimize(algo,
                   rec,
                   N,
                   coef_art=coef_art,
                   norm_all=norm_all,
                   preproc_model=preproc_model,
                   preproc_algo=preproc_algo)
    tup = (mrr, N, norm_all, preproc_model, preproc_algo, coef_art)
    ls_full.append(tup)


--- 131.2954077720642 seconds model_bpr ---
N: 625, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 0.75


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
25000it [18:11, 22.90it/s]


MRR@100 = 0.0330


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(



--- 126.61101365089417 seconds model_bpr ---
N: 500, norm_all: True, preproc_model: standart, preproc_algo: standart, coef_art: 0.5


...

MRR@100 = 0.0317

--- 137.55169439315796 seconds model_bpr ---
N: 625, norm_all: True, preproc_model: minmax, preproc_algo: standart, coef_art: 0.05


25000it [17:07, 24.32it/s]


MRR@100 = 0.0327

--- 129.97491812705994 seconds model_bpr ---
N: 750, norm_all: True, preproc_model: minmax, preproc_algo: standart, coef_art: 0.05


5399it [03:45, 23.97it/s]


KeyboardInterrupt: 